In [1]:
import pandas as pd
import os
import json

## Carga de los corpus y creación del corpus combinado

In [2]:
# Visualizacion inicial de la fuente 2

corpus_fuente_2 = pd.read_csv(r'..\fuente_2\fuente_2.csv', sep=';')

# Eliminamos columnas innecesarias y hacemos drop a las filas con algun valor nulo
corpus_fuente_2 = corpus_fuente_2.drop(columns=['Fecha', 'ID'])
corpus_fuente_2 = corpus_fuente_2.dropna()
corpus_fuente_2 = corpus_fuente_2.rename(columns={'Label':'label', 'Titulo':'titulo', 'Descripcion':'descripcion'})

# Tomamos una muestra balanceada de 7000 filas para usar en el posterior entrenamiento de modelos 
corpus_fuente_2_balanceado = (
    corpus_fuente_2.groupby('label', group_keys=False)
      .apply(lambda x: x.sample(n=3500, random_state=42))
)

corpus_fuente_2_balanceado

C:\Users\Usuario\AppData\Local\Temp\ipykernel_23656\4256392429.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=3500, random_state=42))


,label,titulo,descripcion
18918,0,"El legado del BNG en Madrid: desigualdad, anem...",A lo largo de la campaña se hablará más en cla...
52784,0,Pablo Mónica García pide a David de la Cruz el...,El secretario general de PP lo hizo por mensaj...
4122,0,Pablo Echenique niega haber creado un 'chiring...,La oficina en la que trabajará Irene Lozano ha...
11851,0,Ribó analiza en 'La Base' la victoria de Petro...,El expresidente del Gobierno ha valorado el tr...
4262,0,Carrero Blanco: 'Queremos gobernar poniendo el...,Carrero Blanco quiere que el Ayuntamiento de M...
...,...,...,...
52762,1,PSC y JxCat cierran un acuerdo para gobernar j...,"En el documento, las dos fuerzas acuerdan su p..."
10921,1,PP y Cs lanzan una ofensiva contra Belarra y p...,"La presidenta del partido naranja, Inés Arrima..."
13413,1,El PSOE se crece de cara al 28M ante las 'duda...,La Conferencia Municipal de los socialistas si...
22593,1,Campo defiende a Marlaska: el cese de Pérez de...,El ministro de Justicia descarta la injerencia...


In [3]:
def cargar_corpus_estudiante(carpetas):

    # Lista para ir guardando filas
    rows = []

    # Recorremos carpetas y archivos
    for folder in carpetas:
        for filename in os.listdir(folder):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder, filename)
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)  # cada archivo es un array de objetos JSON
                    for item in data:
                        # extraemos label, title, body
                        label = item.get('label')
                        title = item.get('title')
                        body = item.get('body')
                        rows.append([label, title, body])

    # Creamos DataFrame
    df = pd.DataFrame(rows, columns=['label', 'titulo', 'descripcion'])

    return df


df_corpus_estudiantes = cargar_corpus_estudiante([r'..\fuente_1\corpus_falsas',r'..\fuente_1\corpus_verdaderas'])

# Desordenar todas las filas
df_corpus_estudiantes = df_corpus_estudiantes.sample(frac=1, ignore_index=True)

df_corpus_estudiantes['label'] = df_corpus_estudiantes['label'].astype(int)

df_corpus_estudiantes

,label,titulo,descripcion
0,1,"Henrique Capriles: ""La mayor parte de las pers...",Henrique Capriles ha sido una figura constante...
1,0,Son engañosas las publicaciones que aseguran q...,Circulan en Facebook distintos posteos que ase...
2,1,Por qué el despliegue del ejército de EE.UU. e...,"""No estamos de acuerdo con la presencia milita..."
3,1,Ley 1438,Ministerio de Salud y Protección Social ​Indic...
4,0,Carolina Corcho no trinó que solo tendrán acce...,Falso Falso Falso Falso Falso Falso Falso Fals...
...,...,...,...
195,0,Cáncer no es causado por un hongo ni se cura c...,Falso Falso Falso Falso Falso Falso Falso Fals...
196,1,El modelo de bicis de alquiler se hunde en Bog...,"A la caída del 27,15% en los viajes de la red ..."
197,1,El día que las mujeres se rebelaron contra el ...,"Armadas con piedras, cuchillos, ollas, sartene..."
198,1,Honduras valida su Plan de Acción para elimina...,"Tegucigalpa, 18 de septiembre 2025 (OPS/OMS). ..."


In [4]:
# Unimos los dos dataframes para crear el corpus a utilizar

corpus_unido = pd.concat([df_corpus_estudiantes, corpus_fuente_2_balanceado])

corpus_unido = corpus_unido.sample(frac=1, ignore_index=True)

corpus_unido.to_csv(r'..\1_creacion_corpus\1_corpus_unido.csv')

## Metricas del texto


In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

def extraer_metricas_corpus_nltk(corpus):
    '''
    En esta función se extraen las métricas del corpus final de noticias usando NLTK
    '''
    # 1. Combinar título y descripción
    corpus["text"] = corpus["titulo"].astype(str) + " " + corpus["descripcion"].astype(str)

    # 2. Tokenizar cada documento con NLTK (español)
    corpus["tokens"] = corpus["text"].apply(lambda x: word_tokenize(str(x), language="spanish"))

    # 3. Contar palabras (sin puntuación)
    corpus["word_count"] = corpus["tokens"].apply(lambda tokens: len([t for t in tokens if t.isalpha()]))

    # 4. Calcular métricas
    mean_words = corpus["word_count"].mean()
    std_words = corpus["word_count"].std()
    min_words = corpus["word_count"].min()
    max_words = corpus["word_count"].max()
    quartiles = corpus["word_count"].quantile([0.25, 0.5, 0.75])
    total_words = corpus["word_count"].sum()

    # 5. Mostrar resultados
    print("📊 Métricas sobre la cantidad de palabras (titulo + descripcion) con NLTK:")
    print(f"Mean (promedio): {mean_words:.2f}")
    print(f"Std (desviación estándar): {std_words:.2f}")
    print(f"Min (mínimo): {min_words}")
    print(f"Max (máximo): {max_words}")
    print("Quartiles:")
    print(quartiles)
    print(f"Sum (total de palabras): {total_words}")

    return None

extraer_metricas_corpus_nltk(corpus_unido)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


📊 Métricas sobre la cantidad de palabras (titulo + descripcion) con NLTK:
Mean (promedio): 68.51
Std (desviación estándar): 139.81
Min (mínimo): 16
Max (máximo): 3516
Quartiles:
0.25    43.0
0.50    52.0
0.75    62.0
Name: word_count, dtype: float64
Sum (total de palabras): 493262
